In [1]:
import pandas as pd 
import xgboost 

In [2]:
# allow displaying all rows and columns in pandas outputs (uses pandas imported in earlier cell)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [18]:
df_evaluation=pd.read_csv("../data/sales_train_evaluation.csv")
df_validation=pd.read_csv("../data/sales_train_validation.csv")
df_sellprices=pd.read_csv("../data/sell_prices.csv")
df_calendar=pd.read_csv("../data/calendar.csv")

In [4]:
print(df_evaluation.shape ,
      df_validation.shape,
      df_sellprices.shape,  
      df_calendar.shape)

(30490, 1947) (30490, 1919) (6841121, 4) (1969, 14)


**we will dump the evaluation set for this moment**

In [49]:
df_sellprices.head(10)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
5,CA_1,HOBBIES_1_001,11330,8.26
6,CA_1,HOBBIES_1_001,11331,8.26
7,CA_1,HOBBIES_1_001,11332,8.26
8,CA_1,HOBBIES_1_001,11333,8.26
9,CA_1,HOBBIES_1_001,11334,8.26


In [52]:
df_sellprices['item_id'].nunique()

3049

In [51]:
df_validation['store_id'].nunique()

10

In [5]:
df_validation.head(10)

id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   
5  HOBBIES_1_006_CA_1_validation  HOBBIES_1_006  HOBBIES_1  HOBBIES     CA_1   
6  HOBBIES_1_007_CA_1_validation  HOBBIES_1_007  HOBBIES_1  HOBBIES     CA_1   
7  HOBBIES_1_008_CA_1_validation  HOBBIES_1_008  HOBBIES_1  HOBBIES     CA_1   
8  HOBBIES_1_009_CA_1_validation  HOBBIES_1_009  HOBBIES_1  HOBBIES     CA_1   
9  HOBBIES_1_010_CA_1_validation  HOBBIES_1_010  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  d_11  d_12  \
0       CA    0    0    0    0    0    0    0    0    0     0     0     0   
1       CA    0    0    0    0    0    0    0    0    0     0     0     0   
2       CA    0    0    0    0    0    0    0    0    0     0     0     0   
3       CA    0    0    0    0    0    0    0    0    0     0     0     0   
4       CA    0    0    0    0    0    0    0    0    0     0     0     0   
5       CA    0    0    0    0    0    0    0    0    0     0     0     0   
6       CA    0    0    0    0    0    0    0    0    0     0     0     0   
7       CA   12   15    0    0    0    4    6    5    7     0    13     2   
8       CA    2    0    7    3    0    2    3    9    0     0     4     3   
9       CA    0    0    1    0    0    0    0    0    0     0     0     0   

   d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  d_23  d_24  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   
5     0     0     0     0     0     0     0     0     0     0     0     0   
6     0     0     0     0     0     0     0     0     0     0     0     0   
7     2    20     9     3     0     1     0     0     0    10     4     4   
8     1     0     4     4     1     2     0     1     1     5     2     2   
9     0     2     0     0     0     0     2     0     0     0     1     0   

   d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  d_35  d_36  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   
5     0     0     0     0     0     0     0     0     0     0     0     0   
6     0     0     0     0     0     0     0     0     0     0     0     0   
7     2     6     2     0     0     0     0     0     0     0     0     0   
8     2     3     0     0     8     0     0     0     0     0     2     7   
9     0     0     0     0     0     0     0     0     0     0     0     0   

   d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  d_47  d_48  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     2     0     0     0     2     0     1     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   
5     0     0     0     0     0     0     0     0     0     0     0     0   
6     0     0     0     0     0     0     0     0     0     0     

In [41]:
print(f"num items: {df_validation['item_id'].nunique()} \nnum store :{df_validation['store_id'].nunique()}")

num items: 3049 
num store :10


In [38]:
df_validation['store_id'].value_counts()

store_id
CA_1    3049
CA_2    3049
CA_3    3049
CA_4    3049
TX_1    3049
TX_2    3049
TX_3    3049
WI_1    3049
WI_2    3049
WI_3    3049
Name: count, dtype: int64

In [36]:
df_validation['cat_id'].value_counts()

cat_id
FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: count, dtype: int64

In [35]:
df_validation['dept_id'].value_counts()

dept_id
FOODS_3        8230
HOUSEHOLD_1    5320
HOUSEHOLD_2    5150
HOBBIES_1      4160
FOODS_2        3980
FOODS_1        2160
HOBBIES_2      1490
Name: count, dtype: int64

In [28]:
df_validation['store_id'].nunique()

10

In [19]:
df_validation['item_id'].value_counts()
#each item appears 10 times 

item_id
HOBBIES_1_001      10
HOBBIES_1_002      10
HOBBIES_1_003      10
HOBBIES_1_004      10
HOBBIES_1_005      10
HOBBIES_1_006      10
HOBBIES_1_007      10
HOBBIES_1_008      10
HOBBIES_1_009      10
HOBBIES_1_010      10
HOBBIES_1_011      10
HOBBIES_1_012      10
HOBBIES_1_013      10
HOBBIES_1_014      10
HOBBIES_1_015      10
HOBBIES_1_016      10
HOBBIES_1_017      10
HOBBIES_1_018      10
HOBBIES_1_019      10
HOBBIES_1_020      10
HOBBIES_1_021      10
HOBBIES_1_022      10
HOBBIES_1_023      10
HOBBIES_1_024      10
HOBBIES_1_025      10
HOBBIES_1_026      10
HOBBIES_1_027      10
HOBBIES_1_028      10
HOBBIES_1_029      10
HOBBIES_1_030      10
HOBBIES_1_031      10
HOBBIES_1_032      10
HOBBIES_1_033      10
HOBBIES_1_034      10
HOBBIES_1_035      10
HOBBIES_1_036      10
HOBBIES_1_037      10
HOBBIES_1_038      10
HOBBIES_1_039      10
HOBBIES_1_040      10
HOBBIES_1_041      10
HOBBIES_1_042      10
HOBBIES_1_043      10
HOBBIES_1_044      10
HOBBIES_1_045      10
HO

In [ ]:
df_sellprices.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [30]:
print(df_sellprices.shape)
df_sellprices.head(10)

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
5,CA_1,HOBBIES_1_001,11330,8.26
6,CA_1,HOBBIES_1_001,11331,8.26
7,CA_1,HOBBIES_1_001,11332,8.26
8,CA_1,HOBBIES_1_001,11333,8.26
9,CA_1,HOBBIES_1_001,11334,8.26


In [48]:
df_sellprices['wm_yr_wk'].nunique()

282

In [6]:
df_sellprices['item_id'].nunique()

3049

In [25]:
print(df_calendar.shape)
df_calendar.head()

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [44]:
df_calendar["date"].nunique()

1969

In [46]:
df_calendar['event_name_1'].unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)